### 개선판 word2vec 학습

### CBOW 모델 구현

In [1]:
import sys
sys.path.append('..')
from common.np import *  # import numpy as np
from common.layers import Embedding
from ch04.negative_sampling_layer import NegativeSamplingLoss


class CBOW:
    def __init__(self, vocab_size, hidden_size, window_size, corpus):
        V, H = vocab_size, hidden_size

        # 가중치 초기화
        W_in = 0.01 * np.random.randn(V, H).astype('f')
        W_out = 0.01 * np.random.randn(V, H).astype('f')

        # 계층 생성
        self.in_layers = []
        for i in range(2 * window_size):
            layer = Embedding(W_in)  # Embedding 계층 사용
            self.in_layers.append(layer)
        self.ns_loss = NegativeSamplingLoss(W_out, corpus, power=0.75, sample_size=5)

        # 모든 가중치와 기울기를 배열에 모은다.
        layers = self.in_layers + [self.ns_loss]
        self.params, self.grads = [], []
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads

        # 인스턴스 변수에 단어의 분산 표현을 저장한다.
        self.word_vecs = W_in

    def forward(self, contexts, target):
        h = 0
        for i, layer in enumerate(self.in_layers):
            h += layer.forward(contexts[:, i])
        h *= 1 / len(self.in_layers)
        loss = self.ns_loss.forward(h, target)
        return loss

    def backward(self, dout=1):
        dout = self.ns_loss.backward(dout)
        dout *= 1 / len(self.in_layers)
        for layer in self.in_layers:
            layer.backward(dout)
        return None


In [2]:
# coding: utf-8
import sys
sys.path.append('..')  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
from common.layers import *
from ch04.negative_sampling_layer import NegativeSamplingLoss


class SkipGram:
    def __init__(self, vocab_size, hidden_size, window_size, corpus):
        V, H = vocab_size, hidden_size
        rn = np.random.randn

        # 가중치 초기화
        W_in = 0.01 * rn(V, H).astype('f')
        W_out = 0.01 * rn(V, H).astype('f')

        # 계층 생성
        self.in_layer = Embedding(W_in)
        self.loss_layers = []
        for i in range(2 * window_size):
            layer = NegativeSamplingLoss(W_out, corpus, power=0.75, sample_size=5)
            self.loss_layers.append(layer)

        # 모든 가중치와 기울기를 리스트에 모은다.
        layers = [self.in_layer] + self.loss_layers
        self.params, self.grads = [], []
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads

        # 인스턴스 변수에 단어의 분산 표현을 저장한다.
        self.word_vecs = W_in

    def forward(self, contexts, target):
        h = self.in_layer.forward(target)

        loss = 0
        for i, layer in enumerate(self.loss_layers):
            loss += layer.forward(h, contexts[:, i])
        return loss

    def backward(self, dout=1):
        dh = 0
        for i, layer in enumerate(self.loss_layers):
            dh += layer.backward(dout)
        self.in_layer.backward(dh)
        return None


### CBOW 모델 학습 코드

In [3]:
# coding: utf-8
import sys
sys.path.append('..')
import numpy as np
from common import config
# GPU에서 실행하려면 아래 주석을 해제하세요(CuPy 필요).
# ===============================================
# config.GPU = True
# ===============================================
import pickle
from common.trainer import Trainer
from common.optimizer import Adam
from cbow import CBOW
from skip_gram import SkipGram
from common.util import create_contexts_target, to_cpu, to_gpu
from dataset import ptb


# 하이퍼파라미터 설정
window_size = 5
hidden_size = 100
batch_size = 100
max_epoch = 10

# 데이터 읽기
corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)

contexts, target = create_contexts_target(corpus, window_size)
if config.GPU:
    contexts, target = to_gpu(contexts), to_gpu(target)

# 모델 등 생성
model = CBOW(vocab_size, hidden_size, window_size, corpus)
# model = SkipGram(vocab_size, hidden_size, window_size, corpus)
optimizer = Adam()
trainer = Trainer(model, optimizer)

# 학습 시작
trainer.fit(contexts, target, max_epoch, batch_size)
trainer.plot()

# 나중에 사용할 수 있도록 필요한 데이터 저장
word_vecs = model.word_vecs
if config.GPU:
    word_vecs = to_cpu(word_vecs)
params = {}
params['word_vecs'] = word_vecs.astype(np.float16)
params['word_to_id'] = word_to_id
params['id_to_word'] = id_to_word
pkl_file = 'cbow_params.pkl'  # or 'skipgram_params.pkl'
with open(pkl_file, 'wb') as f:
    pickle.dump(params, f, -1)


| 에폭 1 |  반복 1 / 9295 | 시간 0[s] | 손실 4.16
| 에폭 1 |  반복 21 / 9295 | 시간 1[s] | 손실 4.16
| 에폭 1 |  반복 41 / 9295 | 시간 4[s] | 손실 4.15
| 에폭 1 |  반복 61 / 9295 | 시간 7[s] | 손실 4.12
| 에폭 1 |  반복 81 / 9295 | 시간 10[s] | 손실 4.05
| 에폭 1 |  반복 101 / 9295 | 시간 13[s] | 손실 3.92
| 에폭 1 |  반복 121 / 9295 | 시간 15[s] | 손실 3.78
| 에폭 1 |  반복 141 / 9295 | 시간 17[s] | 손실 3.62
| 에폭 1 |  반복 161 / 9295 | 시간 20[s] | 손실 3.49
| 에폭 1 |  반복 181 / 9295 | 시간 22[s] | 손실 3.34
| 에폭 1 |  반복 201 / 9295 | 시간 24[s] | 손실 3.26
| 에폭 1 |  반복 221 / 9295 | 시간 26[s] | 손실 3.15
| 에폭 1 |  반복 241 / 9295 | 시간 30[s] | 손실 3.08
| 에폭 1 |  반복 261 / 9295 | 시간 32[s] | 손실 3.01
| 에폭 1 |  반복 281 / 9295 | 시간 35[s] | 손실 2.96
| 에폭 1 |  반복 301 / 9295 | 시간 37[s] | 손실 2.92
| 에폭 1 |  반복 321 / 9295 | 시간 39[s] | 손실 2.88
| 에폭 1 |  반복 341 / 9295 | 시간 40[s] | 손실 2.85
| 에폭 1 |  반복 361 / 9295 | 시간 42[s] | 손실 2.82
| 에폭 1 |  반복 381 / 9295 | 시간 44[s] | 손실 2.80
| 에폭 1 |  반복 401 / 9295 | 시간 46[s] | 손실 2.76
| 에폭 1 |  반복 421 / 9295 | 시간 48[s] | 손실 2.76
| 에폭 1 |  반복 441 / 9

| 에폭 1 |  반복 3541 / 9295 | 시간 327[s] | 손실 2.44
| 에폭 1 |  반복 3561 / 9295 | 시간 329[s] | 손실 2.41
| 에폭 1 |  반복 3581 / 9295 | 시간 331[s] | 손실 2.41
| 에폭 1 |  반복 3601 / 9295 | 시간 332[s] | 손실 2.43
| 에폭 1 |  반복 3621 / 9295 | 시간 334[s] | 손실 2.40
| 에폭 1 |  반복 3641 / 9295 | 시간 336[s] | 손실 2.40
| 에폭 1 |  반복 3661 / 9295 | 시간 338[s] | 손실 2.44
| 에폭 1 |  반복 3681 / 9295 | 시간 339[s] | 손실 2.44
| 에폭 1 |  반복 3701 / 9295 | 시간 341[s] | 손실 2.42
| 에폭 1 |  반복 3721 / 9295 | 시간 343[s] | 손실 2.40
| 에폭 1 |  반복 3741 / 9295 | 시간 345[s] | 손실 2.40
| 에폭 1 |  반복 3761 / 9295 | 시간 346[s] | 손실 2.42
| 에폭 1 |  반복 3781 / 9295 | 시간 348[s] | 손실 2.42
| 에폭 1 |  반복 3801 / 9295 | 시간 350[s] | 손실 2.42
| 에폭 1 |  반복 3821 / 9295 | 시간 352[s] | 손실 2.40
| 에폭 1 |  반복 3841 / 9295 | 시간 354[s] | 손실 2.42
| 에폭 1 |  반복 3861 / 9295 | 시간 356[s] | 손실 2.43
| 에폭 1 |  반복 3881 / 9295 | 시간 358[s] | 손실 2.43
| 에폭 1 |  반복 3901 / 9295 | 시간 360[s] | 손실 2.38
| 에폭 1 |  반복 3921 / 9295 | 시간 361[s] | 손실 2.42
| 에폭 1 |  반복 3941 / 9295 | 시간 363[s] | 손실 2.41
| 에폭 1 |  반복 

| 에폭 1 |  반복 7041 / 9295 | 시간 683[s] | 손실 2.30
| 에폭 1 |  반복 7061 / 9295 | 시간 685[s] | 손실 2.27
| 에폭 1 |  반복 7081 / 9295 | 시간 688[s] | 손실 2.29
| 에폭 1 |  반복 7101 / 9295 | 시간 690[s] | 손실 2.28
| 에폭 1 |  반복 7121 / 9295 | 시간 692[s] | 손실 2.31
| 에폭 1 |  반복 7141 / 9295 | 시간 694[s] | 손실 2.27
| 에폭 1 |  반복 7161 / 9295 | 시간 696[s] | 손실 2.26
| 에폭 1 |  반복 7181 / 9295 | 시간 698[s] | 손실 2.26
| 에폭 1 |  반복 7201 / 9295 | 시간 701[s] | 손실 2.27
| 에폭 1 |  반복 7221 / 9295 | 시간 703[s] | 손실 2.27
| 에폭 1 |  반복 7241 / 9295 | 시간 705[s] | 손실 2.30
| 에폭 1 |  반복 7261 / 9295 | 시간 707[s] | 손실 2.27
| 에폭 1 |  반복 7281 / 9295 | 시간 710[s] | 손실 2.31
| 에폭 1 |  반복 7301 / 9295 | 시간 712[s] | 손실 2.28
| 에폭 1 |  반복 7321 / 9295 | 시간 714[s] | 손실 2.29
| 에폭 1 |  반복 7341 / 9295 | 시간 717[s] | 손실 2.30
| 에폭 1 |  반복 7361 / 9295 | 시간 719[s] | 손실 2.28
| 에폭 1 |  반복 7381 / 9295 | 시간 721[s] | 손실 2.27
| 에폭 1 |  반복 7401 / 9295 | 시간 723[s] | 손실 2.28
| 에폭 1 |  반복 7421 / 9295 | 시간 726[s] | 손실 2.28
| 에폭 1 |  반복 7441 / 9295 | 시간 728[s] | 손실 2.29
| 에폭 1 |  반복 

| 에폭 2 |  반복 1261 / 9295 | 시간 1015[s] | 손실 2.17
| 에폭 2 |  반복 1281 / 9295 | 시간 1017[s] | 손실 2.18
| 에폭 2 |  반복 1301 / 9295 | 시간 1019[s] | 손실 2.14
| 에폭 2 |  반복 1321 / 9295 | 시간 1020[s] | 손실 2.15
| 에폭 2 |  반복 1341 / 9295 | 시간 1022[s] | 손실 2.14
| 에폭 2 |  반복 1361 / 9295 | 시간 1024[s] | 손실 2.14
| 에폭 2 |  반복 1381 / 9295 | 시간 1026[s] | 손실 2.11
| 에폭 2 |  반복 1401 / 9295 | 시간 1027[s] | 손실 2.14
| 에폭 2 |  반복 1421 / 9295 | 시간 1029[s] | 손실 2.13
| 에폭 2 |  반복 1441 / 9295 | 시간 1031[s] | 손실 2.15
| 에폭 2 |  반복 1461 / 9295 | 시간 1034[s] | 손실 2.18
| 에폭 2 |  반복 1481 / 9295 | 시간 1036[s] | 손실 2.16
| 에폭 2 |  반복 1501 / 9295 | 시간 1038[s] | 손실 2.14
| 에폭 2 |  반복 1521 / 9295 | 시간 1041[s] | 손실 2.14
| 에폭 2 |  반복 1541 / 9295 | 시간 1043[s] | 손실 2.17
| 에폭 2 |  반복 1561 / 9295 | 시간 1045[s] | 손실 2.16
| 에폭 2 |  반복 1581 / 9295 | 시간 1048[s] | 손실 2.16
| 에폭 2 |  반복 1601 / 9295 | 시간 1050[s] | 손실 2.19
| 에폭 2 |  반복 1621 / 9295 | 시간 1052[s] | 손실 2.16
| 에폭 2 |  반복 1641 / 9295 | 시간 1054[s] | 손실 2.16
| 에폭 2 |  반복 1661 / 9295 | 시간 1056[s] | 

KeyboardInterrupt: 

###  CBOW 모델 평가

In [4]:
# coding: utf-8
import sys
sys.path.append('..')
from common.util import most_similar, analogy
import pickle


pkl_file = 'cbow_params.pkl'
# pkl_file = 'skipgram_params.pkl'

with open(pkl_file, 'rb') as f:
    params = pickle.load(f)
    word_vecs = params['word_vecs']
    word_to_id = params['word_to_id']
    id_to_word = params['id_to_word']

# 가장 비슷한(most similar) 단어 뽑기
querys = ['you', 'year', 'car', 'toyota']
for query in querys:
    most_similar(query, word_to_id, id_to_word, word_vecs, top=5)


[query] you
 we: 0.71533203125
 i: 0.705078125
 your: 0.626953125
 they: 0.6171875
 anything: 0.59716796875

[query] year
 month: 0.83837890625
 week: 0.787109375
 spring: 0.767578125
 summer: 0.759765625
 decade: 0.697265625

[query] car
 truck: 0.6337890625
 luxury: 0.6328125
 auto: 0.5751953125
 window: 0.57421875
 cars: 0.55859375

[query] toyota
 honda: 0.658203125
 marathon: 0.6279296875
 coated: 0.62646484375
 engines: 0.61962890625
 seita: 0.61669921875


In [5]:
# 유추(analogy) 작업
print('-'*50)
analogy('king', 'man', 'queen',  word_to_id, id_to_word, word_vecs)
analogy('take', 'took', 'go',  word_to_id, id_to_word, word_vecs)
analogy('car', 'cars', 'child',  word_to_id, id_to_word, word_vecs)
analogy('good', 'better', 'bad',  word_to_id, id_to_word, word_vecs)

--------------------------------------------------

[analogy] king:man = queen:?
 a.m: 5.7421875
 woman: 5.50390625
 naczelnik: 5.10546875
 carolinas: 4.93359375
 hacker: 4.74609375

[analogy] take:took = go:?
 went: 4.6796875
 're: 4.45703125
 came: 4.4140625
 were: 4.07421875
 was: 4.0625

[analogy] car:cars = child:?
 a.m: 6.75390625
 rape: 5.875
 children: 5.3203125
 daffynition: 4.734375
 women: 4.5390625

[analogy] good:better = bad:?
 rather: 5.78125
 more: 5.73828125
 less: 5.40625
 greater: 4.5078125
 fewer: 4.1328125
